In [1]:
import pandas as pd
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt

# File path
dataset_fortex = '/Users/Erlemico/Downloads/dataset-fortex-2023.csv'

# Set chunk size for processing large dataset
chunk_size = 10000

# Set dtype for column index 15
dtype_dict = {15: str}

# Read dataset in chunks
chunk_iter = pd.read_csv(dataset_fortex, chunksize=chunk_size, low_memory=False, dtype=dtype_dict)

# Combine all chunks into a single DataFrame
df_list = []
for chunk in chunk_iter:
    df_list.append(chunk)

# Merge all the chunks into one big DataFrame
df = pd.concat(df_list, ignore_index=True)

# Show the first few rows
display(HTML(df.head().to_html()))

# Show total number of rows and columns
print(f"Total rows: {df.shape[0]}, Total columns: {df.shape[1]}")

Total rows: 77964, Total columns: 122


In [2]:
# Check data types and null values
df.info()

# Check for missing values
missing_data = df.isnull().sum()
print("Missing values per column:\n", missing_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77964 entries, 0 to 77963
Columns: 122 entries, 1. YEAR to 122. 8.9 - PRODUCTION RATIO
dtypes: float64(77), int64(15), object(30)
memory usage: 72.6+ MB
Missing values per column:
 1. YEAR                               0
2. TRIFD                              0
3. FRS ID                            19
4. FACILITY NAME                      0
5. STREET ADDRESS                     0
                                  ...  
118. 8.7 - TREATMENT OFF SITE         0
119. PRODUCTION WSTE (8.1-8.7)        0
120. 8.8 - ONE-TIME RELEASE       67171
121. PROD_RATIO_OR_ ACTIVITY      10358
122. 8.9 - PRODUCTION RATIO        1594
Length: 122, dtype: int64


In [3]:
# Drop columns where more than 50% of the values are empty
threshold = len(df) * 0.5
df = df.dropna(thresh=threshold, axis=1)

# For numeric columns: fill null with median
num_cols = df.select_dtypes(include='number').columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# For category column: fill null with mode
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

In [4]:
# Encoding Categorical Columns
df_encoded = pd.get_dummies(df, drop_first=True)

In [5]:
# Normalisation of Numerical Data
from sklearn.preprocessing import MinMaxScaler

# Scaler initialisation
scaler = MinMaxScaler()

# Retrieve only numeric columns
num_cols = df_encoded.select_dtypes(include='number').columns

# Normalise
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

In [6]:
# Display all column names (79 in total)
pd.DataFrame(df_encoded.columns.tolist(), columns=["Columns"]).head(79)

Columns
0                          1. YEAR
1                        3. FRS ID
2                           9. ZIP
3                     12. LATITUDE
4                    13. LONGITUDE
..                             ...
74    116. 8.5 - RECYCLING OFF SIT
75    117. 8.6 - TREATMENT ON SITE
76   118. 8.7 - TREATMENT OFF SITE
77  119. PRODUCTION WSTE (8.1-8.7)
78     122. 8.9 - PRODUCTION RATIO

[79 rows x 1 columns]

In [7]:
# Separate targets and features
target_column = '122. 8.9 - PRODUCTION RATIO'
X = df_encoded.drop(columns=[target_column])
y = df_encoded[target_column]

In [8]:
# Feature Selection using SelectKBest + f_regression
from sklearn.feature_selection import SelectKBest, f_regression
import pandas as pd

# SelectKBest initialisation to select the 10 best features
selector = SelectKBest(score_func=f_regression, k=10)
X_new = selector.fit_transform(X, y)

# Retrieve the name of the selected feature and score
selected_features = X.columns[selector.get_support()]
scores = selector.scores_[selector.get_support()]

# Make DataFrame from result of feature selection
feature_scores = pd.DataFrame({
    'Feature': selected_features,
    'Score (F-statistic)': scores
}).sort_values(by='Score (F-statistic)', ascending=False)

print("Top 10 best features based on f_regression:")
print(feature_scores)

Top 10 best features based on f_regression:
                                             Feature  Score (F-statistic)
0                           2. TRIFD_37662MDPPRPOBOX         43648.501869
1  4. FACILITY NAME_DOMTAR PAPER CO LLC KINGSPORT...         43648.501869
2               5. STREET ADDRESS_100 CLINCHFIELD ST         43648.501869
3                                  6. CITY_KINGSPORT          2245.707445
5            17. STANDARD PARENT CO NAME_DOMTAR CORP          1087.784767
4                                 7. COUNTY_SULLIVAN          1071.022680
6  37. CHEMICAL_1,1-Dichloro-1-fluoroethane (HCFC...           530.013884
7               39. TRI CHEMICAL/COMPOUND ID_1717006           530.013884
8                                 40. CAS#_1717-00-6           530.013884
9                                   40. CAS#_94-74-6           251.875233
